In [1]:
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
from pyvo import dal

vizier = dal.TAPService('http://tapvizier.u-strasbg.fr/TAPVizieR/tap')

In [4]:
table_list = vizier.tables

In [5]:
print('number of tables = {:d}'.format(len(table_list)))

number of tables = 37269


In [6]:
tablename = "B/sb9/orbits"
schema = vizier.run_sync('select top 0 * from "{:s}"'.format(tablename)).table
schema

recno,Seq,N,o,Per,f_Per,e_Per,T0,n_T0,e_T0,f_T0,e,f_e,e_e,omega,f_omega,e_omega,K1,u_K1,f_K1,e_K1,K2,u_K2,f_K2,e_K2,V0,u_V0,f_V0,e_V0,rms1,rms2,o_K1,o_K2,Grade,Ref,Contr
,,,,d,,d,d,,d,,,,,deg,,deg,km / s,,,km / s,km / s,,,km / s,km / s,,,km / s,km / s,km / s,,,,,
int32,int16,bytes1,int16,float64,bytes1,float64,float64,bytes1,float64,float64,float64,bytes1,float64,float64,bytes1,float64,float64,bytes1,bytes1,float64,float64,bytes1,bytes1,float64,float64,bytes1,bytes1,float64,float64,float64,int16,int16,float64,object,object


In [7]:
r = vizier.run_sync('select top 10 * from "{tablename}" where Grade>4'.format(tablename=tablename))
r.table[['Seq', 'Grade']]

Seq,Grade
int16,float64
1,5.0
2,1.0
3,4.0
4,4.0
5,2.0
6,2.0
7,1.0
8,3.0
9,3.0


- Expected: get only rows with 'Grade' column above 4
- Outcome: it contains rows with 'Grade' <=4

In [8]:
%%time
# Check number of rows in main table
count_rows_main = """
Select count(sb9_main.RAJ2000)
from "B/sb9/main" sb9_main
"""
r = vizier.run_async(count_rows_main)
print(r.table)

COUNT
-----
 3595
CPU times: user 49.4 ms, sys: 7.68 ms, total: 57.1 ms
Wall time: 8.35 s


In [9]:
%%time
# Check number of rows in orbits table
count_rows_orbits = """
Select count(sb9_orbits.Seq)
from "B/sb9/orbits" sb9_orbits
"""
r = vizier.run_async(count_rows_orbits)
print(r.table)

COUNT
-----
 4506
CPU times: user 51.5 ms, sys: 8.4 ms, total: 59.9 ms
Wall time: 8.37 s


In [10]:
%%time
# I know Seq starts with 1, but check anyway
find_min_seq = """
select min(seq)
from "B/sb9/orbits"
"""
r = vizier.run_async(find_min_seq)
print(r.table)

MIN
---
  1
CPU times: user 54.2 ms, sys: 8.42 ms, total: 62.6 ms
Wall time: 9.91 s


In [11]:
%%time
# I know Seq starts with 1, but check anyway
count_unique_seq = """
select count(distinct seq)
from "B/sb9/orbits"
"""
r = vizier.run_async(count_unique_seq)
print(r.table)

COUNT
-----
 3595
CPU times: user 51.6 ms, sys: 9.21 ms, total: 60.8 ms
Wall time: 9.09 s


In [12]:
%%time
query_join_sb9 = """
SELECT sb9_main.Seq as seq, sb9_main.RAJ2000 as ra, sb9_main.DEJ2000 as dec
FROM "B/sb9/orbits" as sb9_orbits
LEFT JOIN "B/sb9/main" as sb9_main
ON (sb9_orbits.Seq = sb9_main.Seq) and (sb9_orbits.grade>=4)
"""
# because Seq is unique for every row of sb9_orbits,
# I only expect at most the same number of rows as sb_orbits from this LEFT JOIN
r = vizier.run_async(query_join_sb9)
print(r.table)

seq         ra                 dec        
           deg                 deg        
--- ------------------ -------------------
  1 1.3339166666666664 -5.7076111111111105
  2 1.5141249999999997  63.679666666666655
  3 2.0504166666666666 -2.4477222222222217
  4 2.0969166666666665   29.09044444444444
  5 2.2506666666666666  28.247527777777773
  6 2.6752916666666664   54.89149999999999
  7 2.8532499999999996 -11.478638888888888
  8  2.904791666666666  58.211833333333324
  9  3.303041666666666   62.04088888888888
 10  3.745166666666666    8.82097222222222
...                ...                 ...
  0                 --                  --
  0                 --                  --
  0                 --                  --
  0                 --                  --
  0                 --                  --
  0                 --                  --
  0                 --                  --
  0                 --                  --
  0                 --                  --
  0        

* Expected: subset of rows from orbits joined with main
* Outcome:
    - not filtered on Grade
    - what are all those rows with Seq=0?

In [13]:
%%time
query = """
select sub.seq, sub.ra, sub.dec
from (
    SELECT sb9_main.Seq as seq, sb9_main.RAJ2000 as ra, sb9_main.DEJ2000 as dec
    FROM "B/sb9/orbits" as sb9_orbits
    LEFT JOIN "B/sb9/main" as sb9_main
    ON (sb9_orbits.Seq = sb9_main.Seq) and (sb9_orbits.grade>=4)
    ) sub
"""
r = vizier.run_async(query)

CPU times: user 163 ms, sys: 12.4 ms, total: 176 ms
Wall time: 11.1 s


In [14]:
r.table

seq,ra,dec
,deg,deg
int16,float64,float64
1,3.705324074074073e-07,-1.5854475308641972e-06
2,4.2059027777777766e-07,1.768879629629629e-05
3,5.69560185185185e-07,-6.799228395061727e-07
4,5.824768518518517e-07,8.080679012345676e-06
5,6.25185185185185e-07,7.846535493827158e-06
6,7.431365740740739e-07,1.5247638888888884e-05
7,7.925694444444442e-07,-3.1885108024691348e-06
8,8.068865740740738e-07,1.6169953703703698e-05


I'm now just putting the exact same query as a subquery
* Expected: I at least get the same table as the previous query.
* Outcome: RA, Dec values does not make sense

In [15]:
%%time
query = """
SELECT
  "sb9".*, tgas.source_id, tgas.astrometric_excess_noise, tgas.astrometric_excess_noise_sig
FROM
  (
    SELECT sb9_main.Seq as seq, sb9_main.RAJ2000 as ra, sb9_main.DEJ2000 as dec
    FROM "B/sb9/orbits" as sb9_orbits
    LEFT JOIN "B/sb9/main" as sb9_main
    ON sb9_main.Seq = sb9_orbits.Seq and sb9_orbits.grade>=4
  ) sb9, "I/337/tgas" tgas
WHERE
  1=CONTAINS(POINT('ICRS', sb9.ra, sb9.dec), CIRCLE('ICRS', tgas.ra, tgas.dec, 5/3600))
"""
r = vizier.run_async(query)

DALQueryError: Error while processing the ADQL query of the job 1523460774310 !

I checked my query with CDS ADQL validator, and it returns 'Correct'.
* URL: http://cdsportal.u-strasbg.fr/adqltuto/adqlvalidate